# 제2유형_연습하기_iris 종 분류(분류)

# ☑️ 데이터 분석 순서
## 1. 라이브러리 및 데이터 확인
## 2. 데이터 탐색(EDA)
## 3. 데이터 전처리 및 분리
## 4. 모델링 및 성능 평가
## 5. 예측값 제출

# ✅ 1. 라이브러리 및 데이터 확인

In [1]:
import pandas as pd
import numpy as np

In [2]:
############### 실기환경 복사 영역 ###############
import pandas as pd
import numpy as np

# 실기 시험 데이터셋으로 셋팅하기
from sklearn.datasets import load_iris

# Iris 데이터셋 로드
iris = load_iris()
x = pd.DataFrame(iris.data, columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
y = iris.target # 'setosa' = 0, 'versicolor' = 1, 'virginica' = 2
y = np.where(y > 0, 1, 0) # setosa 종은 0, 나머지 종은 1로 변경

# 실기 시험 데이터셋으로 셋팅하기
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, 
                                                    stratify = y, 
                                                    random_state = 2023)

x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
y_train.columns = ['species']

# 결측치 삽입
x_test['sepal_length'].iloc[0] = None
x_train['sepal_length'].iloc[0] = None

# 이상치 삽입
x_train['sepal_width'].iloc[0] = 150
############### 실기환경 복사 영역 ###############

# 🌸 붗꽃의 종(Species)을 분류해보자
## - 데이터의 결측치, 이상치에 대해 처리하고
## - 분류 모델을 사용하여 정확도, F1 score, AUC 값을 산출하시오.
## - 제출은 result 변수에 담아 양식에 맞게 제출하시오.

In [3]:
# 데이터 설명

print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

# ✅ 2. 데이터 탐색(EDA)

In [4]:
# 데이터의 행/열 확인

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(120, 4)
(30, 4)
(120, 1)


In [5]:
# 초기 데이터 확인

print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

    sepal_length  sepal_width  petal_length  petal_width
2            NaN        150.0           1.3          0.2
49           5.0          3.3           1.4          0.2
66           5.6          3.0           4.5          1.5
     sepal_length  sepal_width  petal_length  petal_width
93            NaN          2.3           3.3          1.0
69            5.6          2.5           3.9          1.1
137           6.4          3.1           5.5          1.8
   species
0        0
1        0
2        1


In [6]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 2 to 44
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  119 non-null    float64
 1   sepal_width   120 non-null    float64
 2   petal_length  120 non-null    float64
 3   petal_width   120 non-null    float64
dtypes: float64(4)
memory usage: 4.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 93 to 55
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  29 non-null     float64
 1   sepal_width   30 non-null     float64
 2   petal_length  30 non-null     float64
 3   petal_width   30 non-null     float64
dtypes: float64(4)
memory usage: 1.2 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   species  120 non-

In [7]:
# x_train 과 x_test 데이터의 기초통계량을 비교

print(x_train.describe())
print('-------------------------------------------------------------------------------')
print(x_test.describe())
print('-------------------------------------------------------------------------------')
print(y_train.describe())

       sepal_length  sepal_width  petal_length  petal_width
count    119.000000     120.0000    120.000000   120.000000
mean       5.920168       4.2950      3.816667     1.226667
std        0.841667      13.4191      1.798848     0.780512
min        4.300000       2.2000      1.100000     0.100000
25%        5.150000       2.8000      1.575000     0.300000
50%        6.000000       3.0000      4.400000     1.350000
75%        6.500000       3.4000      5.225000     1.800000
max        7.900000     150.0000      6.900000     2.500000
-------------------------------------------------------------------------------
       sepal_length  sepal_width  petal_length  petal_width
count     29.000000    30.000000     30.000000     30.00000
mean       5.596552     3.000000      3.523333      1.09000
std        0.709367     0.522593      1.631518      0.68549
min        4.600000     2.000000      1.000000      0.10000
25%        5.000000     2.625000      1.600000      0.35000
50%        5.500000 

In [8]:
# y 데이터도 구체적으로 살펴보기

print(y_train.head())

   species
0        0
1        0
2        1
3        1
4        1


In [9]:
# y 데이터도 구체적으로 살펴보기

print(y_train.value_counts())

species
1          80
0          40
Name: count, dtype: int64


# ✅ 3. 데이터 전처리 및 분리

## 1) 결측치 2) 이상치 3) 변수 처리하기

In [10]:
# 결측치 확인

print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64


In [11]:
# 결측치 대체 (중앙값)
# ** 주의사항 : train 데이터의 중앙값으로 test 데이터도 변경해줘야 함 **

median = x_train['sepal_length'].median()
x_train['sepal_length'] = x_train['sepal_length'].fillna(median)
x_test['sepal_length'] = x_test['sepal_length'].fillna(median)

In [13]:
# 이상치 확인

cond1 = (x_train['sepal_width'] >= 10)
print(len(x_train[cond1]))

1


In [15]:
# 이상치 대체
# df['변수명'] = np.where(df['변수명'] >= 5, 대체할 값, df['변수명'])

# 예를 들어 'sepal_width' 값이 10이 넘으면 이상치라고 가정한다면
# 이상치를 제외한 Max 값을 구해서 대체

cond1 = (x_train['sepal_width'] <= 10)
max_sw = x_train[cond1]['sepal_width'].max()
print(max_sw)

x_train['sepal_width'] = np.where(x_train['sepal_width'] >= 10, max_sw, x_train['sepal_width'])
print(x_train.describe())

4.4
       sepal_length  sepal_width  petal_length  petal_width
count    120.000000   120.000000    120.000000   120.000000
mean       5.920833     3.081667      3.816667     1.226667
std        0.838155     0.429966      1.798848     0.780512
min        4.300000     2.200000      1.100000     0.100000
25%        5.175000     2.800000      1.575000     0.300000
50%        6.000000     3.000000      4.400000     1.350000
75%        6.500000     3.400000      5.225000     1.800000
max        7.900000     4.400000      6.900000     2.500000


## 데이터 분리

In [16]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                  y_train['species'], 
                                                  test_size = 0.2, 
                                                  stratify = y_train['species'], 
                                                  random_state = 2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(96, 4)
(24, 4)
(96,)
(24,)


# ✅ 4. 모델링 및 성능 평가

In [17]:
# 랜덤포레스트 모델 사용 (참고 : 회귀 모델은 RandomForestRegressor)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [18]:
# 모델을 사용하여 테스트 데이터 예측

y_pred = model.predict(x_val)

In [19]:
# 모델 성능 평가 (accuracy, f1 score, AUC 등)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
acc = accuracy_score(y_val, y_pred)             # (실제값, 예측값)
f1 = f1_score(y_val, y_pred, average = 'macro') # (실제값, 예측값)
# 다중 분류일 경우 f1 = f1_score(y_val, y_pred, average = 'macro')
auc = roc_auc_score(y_val, y_pred)              # (실제값 예측값)

In [20]:
# 정확도(Accuracy)

print(acc)

1.0


In [21]:
# F1 Score

print(f1)

1.0


In [22]:
# AUC

print(auc)

1.0


In [23]:
# 참고사항
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred) # (실제값, 예측값)
print(cm)

# #### 예측
# #### 0  1
# 실제 0 TN FP
# 실제 1 FN TP

[[ 8  0]
 [ 0 16]]


# ✅ 5. 예측값 제출

## (주의) x_test 를 모델에 넣어 나온 예측값을 제출해야 함

In [24]:
# (실기시험 안내사항)
# 아래 코드 예측 변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'result' : y_result}).to_csv('수험번호.csv', index = False)

# 모델을 사용하여 테스트 데이터 예측

# 1. 특정 클래스로 분류할 경우 (predict)
y_result = model.predict(x_test)
print(y_result[:5])

# 2. 특정 클래스로 분류될 확률을 구할 경우 (predict_proba)
y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

# 이해해보기
result_prob = pd.DataFrame({
    'result' : y_result,
    'prob_0' : y_result_prob[:,0]
})

# setosa 일 확률 : y_result_prob[:,0]
# 그 외 종 일 확률 : y_result_prob[:,1]

print(result_prob[:5])

[1 1 1 0 1]
[[0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.02 0.98]]
   result  prob_0
0       1    0.00
1       1    0.00
2       1    0.00
3       0    1.00
4       1    0.02
